# 오차 역전파
## 미분 계산하기

In [1]:
import torch
import matplotlib.pyplot as plt

def function(x):
    return x**2

def prime_function(x):
    return x*2   # 도함수

x0 = torch.FloatTensor([4])
y0 = prime_function(x0)
print("differentiation of x={} is {:.1f}".format(x0.item(), y0.item()))

differentiation of x=4.0 is 8.0


## 매개변수 개수 구하기
### <역전파 알고리즘의 탄생 배경>
- 경사하강법(Gradient Descent)에서 "미분"이 매개변수를 찾는데 중요하다는 것을 깨달음.
- 매개변수가 "하나"일 경우에는 단순하게 2차원 평면에 그릴 수 있었음.
- h/w 모델의 매개변수 개수가 점점 많아지면... ex. XOR문제도 총 9개 매개변수 필요
- 9개의 변수를 가진 손실 함수로부터 미분을 한번에 계산하는 것은 매우 어려움.
    - --> "역전파 알고리즘" 탄생 !!!

In [2]:
import torch.nn as nn

torch.manual_seed(70)

class Network(nn.Module):
    """ xor network """
    def __init__(self, input_size, hidden_size, output_size):
        super(Network, self).__init__()
        
        self.linear_ih = nn.Linear(in_features=input_size, 
                                   out_features=hidden_size)
        
        self.linear_ho = nn.Linear(in_features=hidden_size, 
                                   out_features=output_size)
        
        self.activation_layer = nn.Sigmoid()
        
    def forward(self, x):
        z1 = self.linear_ih(x)
        a1 = self.activation_layer(z1)
        z2 = self.linear_ho(a1)
        y = self.activation_layer(z2)
        return y
        
net = Network(2,2,1)
num_params = 0

for p in net.parameters():
    num_params += p.view(-1).size(0)

print(num_params)

9


In [11]:
for p in net.parameters():
    print(p)
    print(p.view(-1))
    print(p.view(-1).size()[0])
    print()

Parameter containing:
tensor([[-0.0919, -0.1992],
        [-0.2247, -0.3332]], requires_grad=True)
tensor([-0.0919, -0.1992, -0.2247, -0.3332], grad_fn=<ViewBackward0>)
4

Parameter containing:
tensor([ 0.2342, -0.5175], requires_grad=True)
tensor([ 0.2342, -0.5175], grad_fn=<ViewBackward0>)
2

Parameter containing:
tensor([[0.3162, 0.2359]], requires_grad=True)
tensor([0.3162, 0.2359], grad_fn=<ViewBackward0>)
2

Parameter containing:
tensor([-0.5528], requires_grad=True)
tensor([-0.5528], grad_fn=<ViewBackward0>)
1



### <역전파 알고리즘의 핵심>
- "신경망은 함수의 조합, 각 함수별로 나눠서 미분을 계산하자!"
- 1) 연쇄법칙 2) 계산 그래프

#### 1) 연쇄법칙
- "합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱"

## 2) 계산 그래프 
### z함수 순방향 전파

In [12]:
def z_function(t):
    return t**2

def t_function(x,y):
    return x+y

def forward(x,y):
    t = t_function(x,y)
    z = z_function(t)
    return z

x = torch.Tensor([1])
y = torch.Tensor([3])
z = forward(x,y)
print(z)

tensor([16.])


### z함수 역방향 전파

In [13]:
def z_prime(t):
    return 2*t

def t_prime():
    return 1

def backward(t):
    dx = z_prime(t) * t_prime()
    return dx

x = torch.Tensor([1])
y = torch.Tensor([3])
t = t_function(x,y)
dx = backward(t)
print(dx)

tensor([8.])


## 역전파 수행 과정

In [14]:
"""
torch.Tensor().requires_grad_() 해주면...
계산 그래프상 해당 텐서와 연관된 모든 텐서는
역전파 수행에 관련된 정보를 기록하게 됨! 
"""
x = torch.Tensor([1]).requires_grad_()
y = torch.Tensor([3])
z = forward(x,y) # 두개 더한걸 제곱
# z: tensor([16.], grad_fn=<PowBackward0>)

print("requires grad: x={} y={}, z={}".format(\
        x.requires_grad, y.requires_grad, z.requires_grad))
print("gradient function of z: {}".format(z.grad_fn))
# <PowBackward0 object at 0x000002A50433D250>

# 역전파 수행 버튼
z.backward() # --> 미분값 계산
print("dx = {}".format(x.grad)) # tensor([8.])

requires grad: x=True y=False, z=True
gradient function of z: <PowBackward0 object at 0x000002A50433D250>
dx = tensor([8.])


In [16]:
z.grad
"""
leaf tensor가 아니어서 오류 출력!! 
"""

C:\Users\answl\AppData\Local\Temp\ipykernel_1152\4110045842.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:485.)
  z.grad


In [19]:
y.grad # 출력값 아무것도 없음

In [15]:
z

tensor([16.], grad_fn=<PowBackward0>)

### <역전파 알고리즘의 장점>
- 한번에 계산하던 미분을 여러 구간으로 나눠서 계산함으로써 더 빠르고 효율적인 미분 계산 가능, 미분값이 모델 안에서 어떻게 흘러가는지 알 수 있다.
- 딥러닝 모델을 아주 작은 모듈로 나눌 수 있다.
- PyTorch같은 딥러닝 프레임워크가 역전파를 밑단(백엔드)에서 수행해줌으로써 연구자들이 더 이상 복잡한 수식의 미분을 계산하지 않게 됐다. --> 더 많은 시간을 딥러닝 모델 등 다른 부분에 투자할 수 있게 됨.

### <PyTorch에서 모델을 학습하는 과정>
1. 입력&타겟 데이터 전처리
2. 모델/손실함수/옵티마이저(경사하강법 수행) 정의
3. 에포크 회수 결정 (데이터 첨부터 끝까지 1회 학습)
- 1) 경사 초기화
- 2) 순방향 전파 진행
- 3) 손실값 계산
- 4) 역방향 전파 수행 (오차역전파)
- 5) 옵티마이저로 매개변수 업데이트

# 오차역전파
- 오차 역전파를 위한 도구가 모두 torch.optim 모듈에 있음.
- 확률적 경사하강법 사용 (경사하강법 기반 다양한 알고리즘 존재)

In [20]:
import torch.optim as optim # 오차역전파

# 입력과 타깃 텐서
x = torch.Tensor([[0,0], [0,1], [1,0], [1,1]])
t = torch.Tensor([0,1,1,0])

# XOR 네트워크 정의
net = Network(input_size=2, hidden_size=2, output_size=1)

# 손실함수 정의
loss_function = nn.BCELoss()

# 경사하강법 optimizer 정의: 모델의 매개변수를 전달해줘야함.
optimizer = optim.SGD(params=net.parameters(), lr=0.5)
# 0.5의 학습률로 넷의 매개변수들에 확률적 경사하강법 적용

# 학습 회수 결정 
STEP = 10001

# 학습과정
for step in range(STEP):
    # 1) 경사 초기화
    net.zero_grad()
    
    # 2) 순방향 전파
    y = net(x)
    
    # 3) 손실값 계산
    loss = loss_function(y.squeeze(), t)
    
    # 4) 역방향 전파
    loss.backward()
    
    # 5) optimizer의 경사하강법으로 매개변수 업데이트
    optimizer.step()
    
    if step % 1000 == 0:
        print("{}:\t {:.4f}".format(step, loss.item()))
        
# 올바른 정답을 출력하는지 테스트
pred = net(x).ge(0.5) # greater_equal
print(pred.view(-1))

0:	 0.6927
1000:	 0.2574
2000:	 0.0210
3000:	 0.0102
4000:	 0.0067
5000:	 0.0049
6000:	 0.0039
7000:	 0.0032
8000:	 0.0028
9000:	 0.0024
10000:	 0.0021
tensor([False,  True,  True, False])


torch.ge(n): n보다 크거나 같으면 True, 작으면 False (= torch.greater_equal(n))

torch.gt(n): n보다 크면 True, 작거나 같으면 False

torch.le : 작거나 같으면 True, 크면 False
- torch.less_eqaul 와 동일
- torch.lt : less than

torch.eq : 같으면 True, 다르면 False
- torch.ne 와 반대

torch.equal : 텐서 차원에서 같으면 True, 다르면 False

In [35]:
pred

tensor([[False],
        [ True],
        [ True],
        [False]])

In [22]:
torch.greater_equal(torch.tensor([[1]]), torch.tensor([[0,1], [3,4]]))

tensor([[ True,  True],
        [False, False]])

In [23]:
torch.greater_equal(torch.tensor([[1]]), torch.tensor([0,1, 3,4]))

tensor([[ True,  True, False, False]])

In [24]:
torch.greater_equal(torch.tensor([[1,2,3,4]]), torch.tensor([[0,1], [3,4]]))

RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1

In [25]:
torch.tensor([0,1, 3,4]).shape

torch.Size([4])

In [34]:
a = torch.tensor([[1,2],
             [3,4]])
print(a.shape)
print(a.view(-1).shape)

b = torch.tensor([[1,2,3],
             [3,4,5],
                 [5,6,7]])
print(b.shape)
print(b.view(-1).shape)
print(b)
print(b.view(-1))
print(b.view(3,-1))

torch.Size([2, 2])
torch.Size([4])
torch.Size([3, 3])
torch.Size([9])
tensor([[1, 2, 3],
        [3, 4, 5],
        [5, 6, 7]])
tensor([1, 2, 3, 3, 4, 5, 5, 6, 7])
tensor([[1, 2, 3],
        [3, 4, 5],
        [5, 6, 7]])
